In [1]:
import pandas as pd

# import data
ad = pd.read_csv('./data_ad_all.csv', sep=',', encoding='utf-8', header=None)
no_ad = pd.read_csv('./data_non_ad_all.csv', sep=',', encoding='utf-8', header=None)

# split data
X_ad = ad[1]
y_ad = ad[0]
X_no_ad = no_ad[1]
y_no_ad = no_ad[0]

X_ad.head()

0    힙하다 힙해!부산 버거샵이 압구정로데오로 ~!핫하다 핫해! 오늘은 압구정로데오 맛집...
1    흥안양꼬치051-516-1181부산 금정구 수림로71번길 49(장전동)50m© NA...
2    훗안녕하세요!! 제가 좋아하는 꼬기 꼬기 먹고왔어요!전 웬만하면 고기 퀄리티 때문에...
3    회장님댁오랜만에 서면 1번가에서친구랑 놀다 왔어용 :)역시 하루의 마무리는 alco...
4    회사 점심이 맛이 없어서밖에서 나가서 먹기로 결정!강서브라이트센터 길 건너에 골목 ...
Name: 1, dtype: object

In [3]:
from sklearn.model_selection import train_test_split

# 데이터 분리 
X_ad_train, X_ad_test, y_ad_train, y_ad_test = train_test_split(X_ad, y_ad, test_size=0.1, shuffle=True, random_state=1)
X_no_ad_train, X_no_ad_test, y_no_ad_train, y_no_ad_test = train_test_split(X_no_ad, y_no_ad, test_size=0.1, shuffle=True, random_state=1)

X_train = pd.concat([X_ad_train, X_no_ad_train])
X_test = pd.concat([X_ad_test, X_no_ad_test])
y_train = pd.concat([y_ad_train, y_no_ad_train])
y_test = pd.concat([y_ad_test, y_no_ad_test])
print(X_train.head())
print(X_train.shape)

7587    마장호수출렁다리맛집 죽여주네~안녕하세요, 현블리에요~파주 한우맛집으로 이미 이름을 ...
4288    이번주에 소개할 전국맛집택배는일산 맛집인 더 누룽지의 해물누룽지탕!!해물누룽지탕 2...
4761    잠실새내 맛집 원조부안집 잠실새내점을 소개받아서 친구들이랑 가게 됐어요. 집에서는 ...
4801    잠실 롯데월드몰카페 타르타르에 다녀왔당지하1층 롯데리아 옆! 크록스 옆!세포라 건너...
4228    일산소바맛집 일산돈까스추천 몽화50m© NAVER Corp.몽화경기도 고양시 일산동...
Name: 1, dtype: object
(18343,)


In [4]:
train_data = pd.concat([y_train, X_train], axis=1)
test_data = pd.concat([y_test, X_test], axis=1)

print(train_data.head())
print(test_data.head())
print(train_data.shape)
print(test_data.shape)

       0                                                  1
7587  AD  마장호수출렁다리맛집 죽여주네~안녕하세요, 현블리에요~파주 한우맛집으로 이미 이름을 ...
4288  AD  이번주에 소개할 전국맛집택배는일산 맛집인 더 누룽지의 해물누룽지탕!!해물누룽지탕 2...
4761  AD  잠실새내 맛집 원조부안집 잠실새내점을 소개받아서 친구들이랑 가게 됐어요. 집에서는 ...
4801  AD  잠실 롯데월드몰카페 타르타르에 다녀왔당지하1층 롯데리아 옆! 크록스 옆!세포라 건너...
4228  AD  일산소바맛집 일산돈까스추천 몽화50m© NAVER Corp.몽화경기도 고양시 일산동...
       0                                                  1
9290  AD  사실은 오월 어버이날 이라고 연차내서 여수로 가족여행계획을 세운거였는데요!시골집에 ...
7421  AD  저번주말은 날씨도 좋고 해서 외출이 하고 싶더라구요. 날씨도 곧 바뀌니 쇼핑하러 아...
3847  AD  오늘 소개할 곳은 홍대입구역 8번출구에서 가깝게 위치한 홍대 홍스쭈꾸미 입니다!홍대...
5013  AD  안녕하세요요새 날씨가 너무 춥죠ㅠㅠ면역력 챙기는게 아주 중요하니 다들건강관리 조심하...
3474  AD  경주여행갔다가 최고집사랑하면에서식사도 하고 왔어요 ㅎㅎ제 단골인 최고집사랑하면은MB...
(18343, 2)
(2039, 2)


In [5]:
from konlpy.tag import Mecab
import re

mecab = Mecab()

def mecab_tokenize(text):
    """
        Mecab을 사용하여 tokenized text 반환
    """
    return " ".join(mecab.morphs(text))

def clean_etc_reg_ex(title):
    """
        정규식을 통해 기타 공백과 기호, 숫자등을 제거
    """
    title = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '', title) #remove punctuation
    title = re.sub(r'[∼%①②⑤⑪…→·]', '', title)
    title = re.sub(r'\d+', '', title) #remove number
    title = re.sub(r'\s+', ' ', title) #remove extra space
    title = re.sub(r'<[^>]+>','',title) #remove Html tags
    title = re.sub(r'\s+', ' ', title) #remove spaces
    title = re.sub(r"^\s+", '', title) #remove space from start
    title = re.sub(r'\s+$', '', title) #remove space from the end
    title = re.sub("[一-龥]",'', title)
    return title

def label_to_idx(label, dic):
    return dic[label]

def slice_from_behind(text, num_of_chars):
    return text[-num_of_chars:]


In [6]:
# 특수기호 제거
train_data[1] = train_data[1].apply(str).apply(clean_etc_reg_ex)
test_data[1] = test_data[1].apply(str).apply(clean_etc_reg_ex)

# label to idx
train_data[0] = train_data[0].apply(label_to_idx, dic={"AD":"1", "NO_AD":"0"})
test_data[0] = test_data[0].apply(label_to_idx, dic={"AD":"1", "NO_AD":"0"})

# mecab tokenize
train_data[1] = train_data[1].apply(str).apply(mecab_tokenize)
test_data[1] = test_data[1].apply(str).apply(mecab_tokenize)

# SLICE
train_data[1] = train_data[1].apply(str).apply(slice_from_behind, num_of_chars=500)
test_data[1] = test_data[1].apply(str).apply(slice_from_behind, num_of_chars=500)

test_data.head()

,0,1
9290,1,없 는 양념 게딱지 드 시 려면 그쵸 여수 게장 맛집 여 수낭 만항 게장 으로 오 ...
7421,1,자 로운 양이 였 고 로제 파스타 도 소스 가 훌륭 했 던 곳 이 에요 더 티 트렁...
3847,1,콩나물 Loading 화질 선택 옵션 홍대 가 성비 맛집 홍대 홍 스 쭈꾸미영상으...
5013,1,이 라면 한 판 은 거뜬히 먹 겠 던데요 물론 다이어트 는 잊 고 말 이 죠 D 달...
3474,1,lovenudlemodooat 최고 집사 랑 하 면 네이버 방문자 리뷰 ★ 깊이 가...


In [7]:
# to tsv
train_data.to_csv('train.clean.tok.slice.tsv', sep='\t', encoding='utf-8', index=False, header=None)
test_data.to_csv('test.clean.tok.slice.tsv', sep='\t', encoding='utf-8', index=False, header=None)